Link to Datatset

https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

TFX Introduction follow through. Original project link
https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb#scrollTo=4fqV54CIR6Pu

In [7]:
import tensorflow as tf
from tfx.examples import chicago_taxi_pipeline
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_transform as tft

import os, tempfile, absl, urllib
# import taxi_constants # TAXI_CONSTANTS contains the preprocessing_fn

Setup the pipeline paths

In [8]:
# this is the dir containing the TFX chicago example
_taxi_root = chicago_taxi_pipeline.__path__[0]

# path to push model for serving
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/taxi_simple')

# set up logging
absl.logging.set_verbosity(absl.logging.INFO)

In [9]:
# download the dataset
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, 'data.csv')
urllib.request.urlretrieve(DATA_PATH, _data_filepath)


('C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-data1v5t0tba\\data.csv',
 <http.client.HTTPMessage at 0x20ce4d507f0>)

In [11]:
# initialize the Interactive-Context
context = InteractiveContext()


Ingest the data

In [13]:
# ingests the data
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data C:\Users\DELL\AppData\Local\Temp\tfx-data1v5t0tba\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\CsvExampleGen\\examples\\1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1640608646,sum_checksum:1640608646"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [14]:
# inspect the artifact of EXAMPLE_GEN
# the Data is already spit into TRAIN & EVAL
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)


["train", "eval"] C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\CsvExampleGen\examples\1


Analyze the data

In [16]:
# the output of STEP 1 is the input for this
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\StatisticsGen\statistics\2\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\StatisticsGen\\statistics\\2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
# visualize the analysis provided by the Stats Gen
context.show(statistics_gen.outputs['statistics'])

Genrate the Schema_Gen which is useful to benchmark current model stats for comparison

In [19]:

schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\SchemaGen\schema\3\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\SchemaGen\\schema\\3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [20]:
# visualize the generated stats
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'dropoff_census_tract',INT,required,,-
'dropoff_community_area',INT,required,,-
'dropoff_latitude',FLOAT,required,,-
'dropoff_longitude',FLOAT,required,,-
'fare',FLOAT,required,single,-
'payment_type',STRING,required,single,'payment_type'
'pickup_census_tract',INT,required,,-
'pickup_community_area',INT,required,,-


C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_data_validation\utils\display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '1085 - 72312 N and W Cab Co', '2192 - 73487 Zeymane Corp', '2733 - 74600 Benny Jona', '3011 - 66308 JBL Cab Inc.', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '4053 - 40193 Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '585 - 88805 Valley Cab Co', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation', '0694 - 59280 Chinesco Trans Inc', '2092 - 61288 Sbeih company', '2192 - Zeymane Corp', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3094 - 24059 G.L.B. Cab Co', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '5006 - Salifu Bawa', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6743 - Luhak Corp'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown', 'Prcard'"


Validate that the new Data is on par with defined Schema

In [23]:
# looks for anomalies and null values in the dataset
# takes Statistics and Schema as inputs
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], 
                    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\ExampleValidator\anomalies\4\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\ExampleValidator\anomalies\4\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\ExampleValidator\\anomalies\\4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [24]:
# visualize the validator
context.show(example_validator.outputs['anomalies'])

C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_data_validation\utils\display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Transform with TF-Transform, which does the feature/data engineering

In [33]:
# transform Preprocessing function

_taxi_constants_module_file = 'taxi_constants.py'

%%writefile {_taxi_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]
CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
    'payment_type',
    'company',
]

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'


# this is the PREPROCESSING_FN that takes raw data as input, and return raw inputs as features
_taxi_transform_module_file = 'taxi_transform.py'

%%writefile {_taxi_transform_module_file}

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = taxi_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
        outputs[key] = tft.scale_to_z_score(
            _fill_in_missing(inputs[key])
        )

    for key in _VOCAB_FEATURE_KEYS:
        # Build a vocabulary for this feature.
        outputs[key] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key]), top_k=_VOCAB_SIZE, 
            num_oov_buckets=_OOV_SIZE
        )

    for key in _BUCKET_FEATURE_KEYS:
        outputs[key] = tft.bucketize(_fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT)

    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[key] = _fill_in_missing(inputs[key])

    # was this passenger a big tipper?
    taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
    tips = _fill_in_missing(inputs[_LABEL_KEY])
    outputs[_LABEL_KEY] = tf.where(
        tf.math.is_nan(taxi_fare), tf.cast(tf.zeros_like(taxi_fare), tf.int64),
        # test if the tip was > 20% of the fare.
        tf.cast(tf.greater(tips, tf.multiply(taxi_fare, tf.constants(0.2))), tf.int64)
    ) 
    return outputs



def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
        x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
        in the second dimension.
    Returns:
        A rank 1 tensor where missing values of `x` have been filled in.
    """
    if not isinstance(x, tf.sparse.SparseTensor):
        return x

    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value
        ), axis=1
    )


UsageError: Line magic function `%%writefile` not found.


In [38]:
# takes the Example Gen, Schema Gen and User Defined Code that does the transformation
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("taxi_constants.py")
)

context.run(transform)
# outputs the Transform graph and the Transformed examples

INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\DELL\\Desktop\\Learning projects\\Learning ML pipelines\\taxi_constants.py' (including modules: ['ML_pipelines', 'taxi_constants']).
INFO:absl:User module package has hash fingerprint version 0b6ee190bd3ed33097a596f51421518cb015ea5aa16cbc2902194f8f49fc710d.
INFO:absl:Executing: ['C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python37\\python.exe', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmpsmh54ubl\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmpympf5dzj', '--dist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmpi7rodzjl']
INFO:absl:Successfully built user code wheel distribution at 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\_wheels\\tfx_user_code_Transform-0.0+0b6ee190bd3ed33097a596f51421518cb015ea5aa16cbc2902194f8f49fc710d-py3-none-any.whl'; target user module is 'taxi_constants'.
INFO:absl:Full user module path is 'ta

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature company has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_census_tract has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_community_area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_latitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_longitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature payment_type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_census_tract has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_community_area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_latitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_longitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature tips has no shape. Setting to VarLenSparseTensor.
INFO:absl:Fea

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\Transform\transform_graph\7\.temp_path\tftransform_tmp\0de2b08727054189ab815f7d23ca0bda\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\Transform\transform_graph\7\.temp_path\tftransform_tmp\0de2b08727054189ab815f7d23ca0bda\assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 21
        type_id: 22
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\transform_graph\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 22
        type_id: 14
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\transformed_examples\\7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 23
        type_id: 23
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\updated_analyzer_cache\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 24
        type_id: 18
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\pre_transform_schema\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            str

In [39]:
transform.outputs

{'transform_graph': Channel(
     type_name: TransformGraph
     artifacts: [Artifact(artifact: id: 21
 type_id: 22
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\transform_graph\\7"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.3.0"
   }
 }
 state: LIVE
 , artifact_type: id: 22
 name: "TransformGraph"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'transformed_examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 22
 type_id: 14
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2021-12-27T13_41_14.965748-dk53jxlx\\Transform\\transformed_examples\\

In [40]:
# taking a peek at the transformed artifacts
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

# metadata contains the Schema of the original data
# transformed_metadata contains the Schema of the preprocessed data
# transformed_fn contains the actual preprocessing graph

['metadata', 'transformed_metadata', 'transform_fn']